In [5]:
#FOR FIXED OBSTACLE
import pygame
import numpy as np
import random

# Q-learning parameters
GRID_SIZE = 7
START = (0, 0)
GOAL = (4, 4)
OBSTACLE = [(2, 2), (3, 2)]

ACTIONS = {
    0: (-1, 0),  # Up
    1: (1, 0),   # Down
    2: (0, -1),  # Left
    3: (0, 1)    # Right
}
NUM_ACTIONS = len(ACTIONS)

Q_table = np.zeros((GRID_SIZE, GRID_SIZE, NUM_ACTIONS))
alpha = 0.1
gamma = 0.9
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.1
episodes = 500
max_steps = 100

def is_valid(x, y):
    return 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE and (x, y) not in OBSTACLE

# Train Q-table
for episode in range(episodes):
    x, y = START
    steps = 0
    while (x, y) != GOAL and steps < max_steps:
        steps += 1
        if random.uniform(0, 1) < epsilon:
            action = random.choice(list(ACTIONS.keys()))
        else:
            action = np.argmax(Q_table[x, y])
        new_x = x + ACTIONS[action][0]
        new_y = y + ACTIONS[action][1]
        if is_valid(new_x, new_y):
            reward = 10 if (new_x, new_y) == GOAL else -1
            old_value = Q_table[x, y, action]
            next_max = np.max(Q_table[new_x, new_y])
            Q_table[x, y, action] = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            x, y = new_x, new_y
        else:
            reward = -5
            Q_table[x, y, action] = (1 - alpha) * Q_table[x, y, action] + alpha * reward
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

# Get optimal path
def get_optimal_path():
    x, y = START
    path = [START]
    steps = 0
    max_path_steps = GRID_SIZE * 2
    while (x, y) != GOAL and steps < max_path_steps:
        steps += 1
        action = np.argmax(Q_table[x, y])
        new_x = x + ACTIONS[action][0]
        new_y = y + ACTIONS[action][1]
        if not is_valid(new_x, new_y):
            break
        x, y = new_x, new_y
        path.append((x, y))
    return path

# --- Pygame Visualization ---
pygame.init()
CELL_SIZE = 100
SCREEN_SIZE = GRID_SIZE * CELL_SIZE
screen = pygame.display.set_mode((SCREEN_SIZE, SCREEN_SIZE))
pygame.display.set_caption("Q-Learning Robot Path")

# Colors
WHITE = (255, 255, 255)
GRAY = (200, 200, 200)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
ORANGE = (255, 165, 0)

# Load robot image
robot_img = pygame.image.load("D:/robo.png")
robot_img = pygame.transform.scale(robot_img, (CELL_SIZE, CELL_SIZE))

def draw_grid(path, robot_pos):
    screen.fill(WHITE)
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            rect = pygame.Rect(j * CELL_SIZE, i * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(screen, GRAY, rect, 1)

    for obs in OBSTACLE:
        pygame.draw.rect(screen, RED, (obs[1]*CELL_SIZE, obs[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE))

    pygame.draw.rect(screen, GREEN, (START[1]*CELL_SIZE, START[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE))
    pygame.draw.rect(screen, BLUE, (GOAL[1]*CELL_SIZE, GOAL[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE))

    for (x, y) in path:
        pygame.draw.rect(screen, ORANGE, (y*CELL_SIZE, x*CELL_SIZE, CELL_SIZE, CELL_SIZE))

    screen.blit(robot_img, (robot_pos[1]*CELL_SIZE, robot_pos[0]*CELL_SIZE))
    pygame.display.update()

# Animation loop
optimal_path = get_optimal_path()
speed = 2
max_fps = 10
min_fps = 1
looping = False
clock = pygame.time.Clock()

running = True
frame_index = 0
trail = []

while running:
    clock.tick(speed)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                speed = min(max_fps, speed + 1)
            elif event.key == pygame.K_DOWN:
                speed = max(min_fps, speed - 1)
            elif event.key == pygame.K_l:
                looping = not looping

    if frame_index < len(optimal_path):
        pos = optimal_path[frame_index]
        trail.append(pos)
        draw_grid(trail, pos)
        frame_index += 1
    elif looping:
        frame_index = 0
        trail = []
    else:
        pygame.display.set_caption(f"Q-Learning Robot Path - Finished (Speed: {speed} FPS)")

    pygame.display.set_caption(f"Q-Learning Robot Path - Speed: {speed} FPS")

pygame.quit()

In [6]:
#FOR CHANGING OBSTACLE IN EVERY LOOP
import pygame
import numpy as np
import random
import time

# Q-learning parameters
GRID_SIZE = 8
START = (0, 0)
GOAL = (7,7)

# Function to generate dynamic obstacles
def generate_obstacles(num=3):
    obs = set()
    while len(obs) < num:
        pos = (random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1))
        if pos != START and pos != GOAL:
            obs.add(pos)
    return list(obs)

OBSTACLE = generate_obstacles()

ACTIONS = {
    0: (-1, 0),  # Up
    1: (1, 0),   # Down
    2: (0, -1),  # Left
    3: (0, 1)    # Right
}
NUM_ACTIONS = len(ACTIONS)

Q_table = np.zeros((GRID_SIZE, GRID_SIZE, NUM_ACTIONS))
alpha = 0.1
gamma = 0.9
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.1
episodes = 500
max_steps = 100

def is_valid(x, y):
    return 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE and (x, y) not in OBSTACLE

# Train Q-table
for episode in range(episodes):
    x, y = START
    steps = 0
    while (x, y) != GOAL and steps < max_steps:
        steps += 1
        if random.uniform(0, 1) < epsilon:
            action = random.choice(list(ACTIONS.keys()))
        else:
            action = np.argmax(Q_table[x, y])
        new_x = x + ACTIONS[action][0]
        new_y = y + ACTIONS[action][1]
        if is_valid(new_x, new_y):
            reward = 10 if (new_x, new_y) == GOAL else -1
            old_value = Q_table[x, y, action]
            next_max = np.max(Q_table[new_x, new_y])
            Q_table[x, y, action] = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            x, y = new_x, new_y
        else:
            reward = -5
            Q_table[x, y, action] = (1 - alpha) * Q_table[x, y, action] + alpha * reward
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

# Get optimal path
def get_optimal_path():
    x, y = START
    path = [START]
    steps = 0
    max_path_steps = GRID_SIZE * 2
    while (x, y) != GOAL and steps < max_path_steps:
        steps += 1
        action = np.argmax(Q_table[x, y])
        new_x = x + ACTIONS[action][0]
        new_y = y + ACTIONS[action][1]
        if not is_valid(new_x, new_y):
            break
        x, y = new_x, new_y
        path.append((x, y))
    return path

# --- Pygame Visualization ---
pygame.init()
CELL_SIZE = 100
SCREEN_SIZE = GRID_SIZE * CELL_SIZE
screen = pygame.display.set_mode((SCREEN_SIZE, SCREEN_SIZE))
pygame.display.set_caption("Q-Learning Robot Path")

# Colors
WHITE = (255, 255, 255)
GRAY = (200, 200, 200)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
ORANGE = (255, 165, 0)

# Load robot image
robot_img = pygame.image.load("D:/robo.png")
robot_img = pygame.transform.scale(robot_img, (CELL_SIZE, CELL_SIZE))

def draw_grid(path, robot_pos):
    screen.fill(WHITE)
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            rect = pygame.Rect(j * CELL_SIZE, i * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(screen, GRAY, rect, 1)

    for obs in OBSTACLE:
        pygame.draw.rect(screen, RED, (obs[1]*CELL_SIZE, obs[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE))

    pygame.draw.rect(screen, GREEN, (START[1]*CELL_SIZE, START[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE))
    pygame.draw.rect(screen, BLUE, (GOAL[1]*CELL_SIZE, GOAL[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE))

    for (x, y) in path:
        pygame.draw.rect(screen, ORANGE, (y*CELL_SIZE, x*CELL_SIZE, CELL_SIZE, CELL_SIZE))

    screen.blit(robot_img, (robot_pos[1]*CELL_SIZE, robot_pos[0]*CELL_SIZE))
    pygame.display.update()

# Animation loop
optimal_path = get_optimal_path()
speed = 2
max_fps = 10
min_fps = 1
looping = False
clock = pygame.time.Clock()

# Timer for obstacle change
last_obstacle_change = time.time()
obstacle_change_interval = 8 # seconds

running = True
frame_index = 0
trail = []

while running:
    clock.tick(speed)
    current_time = time.time()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                speed = min(max_fps, speed + 1)
            elif event.key == pygame.K_DOWN:
                speed = max(min_fps, speed - 1)
            elif event.key == pygame.K_l:
                looping = not looping

    # Change obstacles every few seconds
    if current_time - last_obstacle_change > obstacle_change_interval:
        OBSTACLE = generate_obstacles()
        optimal_path = get_optimal_path()
        frame_index = 0
        trail = []
        last_obstacle_change = current_time

    # Recalculate the optimal path on each frame and avoid obstacles
    if frame_index < len(optimal_path):
        pos = optimal_path[frame_index]
        trail.append(pos)
        if not is_valid(pos[0], pos[1]):  # If the robot encounters an obstacle
            # Find a new path immediately (it will try to follow the best action in real-time)
            optimal_path = get_optimal_path()  # Recalculate the path dynamically
            trail = [START]  # Reset the trail
            frame_index = 0  # Restart from the beginning
        draw_grid(trail, pos)
        frame_index += 1
    elif looping:
        frame_index = 0
        trail = []
    else:
        pygame.display.set_caption(f"Q-Learning Robot Path - Finished (Speed: {speed} FPS)")

    pygame.display.set_caption(f"Q-Learning Robot Path - Speed: {speed} FPS")

pygame.quit()